In [1]:
import pandas as pd
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

In [3]:
import pandas as pd

data = pd.read_excel("C:\\Users\\LENOVO\\Downloads\\US_addresses.xlsx")


In [ ]:
# OpenAI API Key (Replace with your actual key)
openai_api_key = ""

In [5]:
# Initialize OpenAI Model
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openai_api_key, temperature=0)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14524\1044613136.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openai_api_key, temperature=0)


In [6]:
system_message = SystemMessage(
    content="""
You are an expert in Indian geography. Your task is to find the **correct pincode** for a given address.
The address contains a **street name, location, and place**.

**Example Inputs & Outputs:**
    Address: "MG Road, Bangalore, Karnataka" → **Pincode: 560001**
    Address: "Sector 15, Gurgaon, Haryana" → **Pincode: 122001**
    Address: "Connaught Place, Delhi, Delhi" → **Pincode: 110001**

 **Rules:**
- Extract only the **6-digit** pincode.
- Do **not** make up a random pincode.
- If unsure, return `"000000"`.

Here is the given address:
"""
)


In [7]:
# Function to extract only the pincode from the response
def extract_pincode(response):
    match = re.search(r'\b\d{6}\b', response)  # Extracts valid 6-digit pincode
    return match.group(0) if match else '000000'  # Default if no pincode found

In [8]:
# Function to fill missing pincodes
def fill_missing_pincodes(row):
    if pd.isna(row['Pincode']) or row['Pincode'] == '':
        address = f"{row['Street Name']}, {row['Location']}, {row['Place']}"
        
        # Human message for the prompt
        human_message = HumanMessage(content=f"Find the correct pincode for based on this address: {address}")
        
        try:
            response = llm([system_message, human_message]).content  # Get the response from OpenAI
            pincode = extract_pincode(response)
            return pincode
        except Exception as e:
            print(f"Error fetching pincode for {address}: {e}")
            return '000000'  # Default for failures
    else:
        return row['Pincode']

In [9]:
# Apply function to dataframe with a delay to avoid rate limits
for i in range(len(data)):
    data.loc[i, 'Pincode'] = fill_missing_pincodes(data.iloc[i])
    time.sleep(1)  # Prevent hitting API limits

In [10]:


# Ensure Pincode is clean before conversion
def clean_pincode(value):
    try:
        return int(float(str(value).strip()))  # Convert to float first, then integer
    except ValueError:
        return None  # Keep it as None if conversion fails

# Apply cleaning function to Pincode column
data['Pincode'] = data['Pincode'].apply(clean_pincode)

# Save the cleaned dataset
data.to_csv('updated_dataset.csv', index=False)

print("Updated dataset saved successfully with integer pincodes.")


Updated dataset saved successfully with integer pincodes.
